In [ ]:
from IPython.display import display, HTML

display(HTML("""
<style>
    .header {
        background: linear-gradient(to right, #003366, #3399ff);
        padding: 20px;
        color: white;
        font-size: 24px;
        font-weight: bold;
        border-radius: 10px;
        display: flex;
        justify-content: space-between;
        align-items: center;
    }
    .framework-title {
        font-size: 22px;
        margin-left: 10px;
    }
    .language-selector {
        font-size: 16px;
        display: flex;
        align-items: center;
    }
    .language-selector select {
        margin-left: 5px;
        padding: 5px 10px;
        font-size: 14px;
    }
    .container {
        flex: 1;
        display: flex;
        flex-direction: column;
        justify-content: space-between;
        padding: 40px 20px;
        background: white;
        border-radius: 10px;
        margin-top: 10px;
    }
    .description {
        font-size: 18px;
        margin-bottom: 30px;
        color: #333;
    }
    .start-button, .nav-button {
        background-color: #1a66b3;
        color: white;
        border: none;
        padding: 15px 30px;
        font-size: 16px;
        border-radius: 10px;
        cursor: pointer;
        margin: 10px 0;
        width: 100%;
    }
    .start-button:hover, .nav-button:hover {
        background-color: #039be5;
    }
    .question-block {
        text-align: center;
    }
    .nav-button-container {
        display: flex;
        justify-content: space-between;
        width: 100%;
    }
    .full-width {
        width: 100%;
    }

    .radial-container {
    display: flex;
    flex-direction: column;
    align-items: center;
    margin-top: 20px;
}
.radial-progress {
    width: 200px;
    height: 100px;
    border-top-left-radius: 100px;
    border-top-right-radius: 100px;
    background: #eee;
    position: relative;
    overflow: hidden;
    margin-bottom: 10px;
}
.radial-fill {
    position: absolute;
    width: 100%;
    height: 100%;
    background: conic-gradient(#1a66b3 calc(var(--progress) * 1%), #eee 0%);
    clip-path: polygon(0% 100%, 0% 0%, 100% 0%, 100% 100%);
    transform: rotate(180deg);
}
.radial-percent {
    font-size: 24px;
    font-weight: bold;
    color: #1a66b3;
}
</style>

<div class="header">
    <div class="framework-title" id="header-title">Digital Maturity Framework</div>
    <div class="language-selector">
        🌐
        <select id="lang-select" onchange="switchLanguage()">
            <option value="pt" selected>Português</option>
            <option value="en">English</option>
        </select>
    </div>
</div>

<div class="container" id="welcome-page">
    <div class="description" id="welcome-text">
        Bem-vindo ao Avaliador de Maturidade Digital.<br>
        Clique no botão abaixo para começar.
    </div>
    <button class="start-button" id="start-button" onclick="startAssessment()">Começar Avaliação</button>
</div>

<div class="container" id="question-page" style="display:none;">
    <div class="question-block">
        <h2 id="question-title"></h2>
        <div id="question-options"></div>
    </div>
    <div class="nav-button-container">
        <button id="back-button" class="nav-button" onclick="goBack()">Back</button>
        <button id="next-button" class="nav-button" onclick="handleNext()">Next</button>
    </div>
</div>

<div class="container" id="result-page" style="display:none;">
    <div class="description" id="result-text"></div>
</div>

<script>
let currentPage = 0;
let answers = {};
let currentLang = 'pt';

const translations = {
    pt: {
        title: 'Avaliador de Maturidade Digital',
        welcome: 'Bem-vindo ao Avaliador de Maturidade Digital.<br>Clique no botão abaixo para começar.',
        start: 'Começar Avaliação',
        next: 'Próximo',
        back: 'Voltar',
        alert: 'Por favor, selecione uma opção.',
        low: 'Maturidade Digital Baixa',
        medium: 'Maturidade Digital Média',
        high: 'Maturidade Digital Elevada',
        recommendations: 'Recomendações:',
        "null": 'Pontuação inválida ou não calculável'
    },
    en: {
        title: 'Digital Maturity Framework',
        welcome: 'Welcome to the Digital Maturity Assessment.<br>Click the button below to start.',
        start: 'Start Assessment',
        next: 'Next',
        back: 'Back',
        alert: 'Please select an option.',
        low: 'Low Digital Maturity',
        medium: 'Medium Digital Maturity',
        high: 'High Digital Maturity',
        recommendations: 'Recommendations:',
        "null": 'Invalid or uncalculated score'
    }
};


function switchLanguage() {
    currentLang = document.getElementById("lang-select").value;
    document.getElementById("header-title").innerHTML = translations[currentLang].title;
    document.getElementById("welcome-text").innerHTML = translations[currentLang].welcome;
    document.getElementById("start-button").innerHTML = translations[currentLang].start;
    document.getElementById("next-button").innerHTML = translations[currentLang].next;
    document.getElementById("back-button").innerHTML = translations[currentLang].back;
}

function startAssessment() {
    document.getElementById("welcome-page").style.display = "none";
    document.getElementById("question-page").style.display = "flex";
    updatePage();
}

function goBack() {
    if (currentPage > 0) {
        currentPage--;
        updatePage();
    }
}

function handleNext() {
    const q = formQuestions[currentPage];
    let selectedValues = [];

    if (q.multi) {
        const checkedOptions = document.querySelectorAll('input[name="option"]:checked');
        if (checkedOptions.length === 0) {
            alert(translations[currentLang].alert);
            return;
        }
        checkedOptions.forEach(opt => selectedValues.push(opt.value));
        answers[`q${currentPage}`] = selectedValues;
    } else {
        const selected = document.querySelector('input[name="option"]:checked');
        if (!selected) {
            alert(translations[currentLang].alert);
            return;
        }
        answers[`q${currentPage}`] = selected.value;
    }

    currentPage++;
    if (currentPage < formQuestions.length) {
        updatePage();
    } else {
        showResult();
    }
}


function updatePage() {
    const q = formQuestions[currentPage];
    let html = '';
    q.options.forEach((opt, i) => {
        html += `<div><input type="${q.multi ? "checkbox" : "radio"}" name="option" value="${opt.value}" id="opt${i}">
                 <label for="opt${i}">${opt.text[currentLang]}</label></div>`;
    });
    document.getElementById("question-title").innerText = q.text[currentLang];
    document.getElementById("question-options").innerHTML = html;  // <-- this was missing
}


function showResult() {
    document.getElementById("question-page").style.display = "none";
    document.getElementById("result-page").style.display = "flex";

    let total = 0;
    Object.entries(answers).forEach(([key, val]) => {
        if (Array.isArray(val)) {
            val.forEach(v => total += parseInt(v));
        } else {
            total += parseInt(val);
        }
    });

    const maxScore = 108;
    const percentage = Math.round((total / maxScore) * 100);

    let message = '';
    if (total < 15) {
        message = translations[currentLang].null;
    } else if (total < 50) {
        message = translations[currentLang].low;
    } else if (total < 80) {
        message = translations[currentLang].medium;
    } else {
        message = translations[currentLang].high;
    }

    let recommendations = [];
    if (total < 15 ) {
        recommendations.push(currentLang === 'pt' ? `Com base nas suas respostas, identificámos que a sua empresa ainda se encontra numa fase inicial de maturidade digital. Isso significa que, neste momento, os seus processos são maioritariamente manuais, com pouca ou nenhuma presença online estruturada.

A nossa recomendação é começar com passos simples, mas essenciais. A primeira prioridade deve ser criar uma presença digital básica — por exemplo, um website institucional, uma página Google Business e perfis profissionais em redes sociais como o Facebook ou o LinkedIn. Estes elementos aumentam a visibilidade e credibilidade da sua empresa.

Além disso, sugerimos que comece a utilizar ferramentas de email corporativo e produtividade na cloud, como o Gmail com Google Workspace ou o Outlook com Microsoft 365. Estas plataformas permitem organizar melhor o trabalho e facilitam a comunicação com clientes e fornecedores.

São mudanças acessíveis, com impacto imediato e baixo custo, que ajudarão a criar as bases para uma transformação digital sólida e progressiva.` : 
`Based on your answers, we identified that your company is currently at a very early stage of digital maturity. This means that your processes are still mostly manual, with little or no structured digital presence.

Our recommendation is to start with simple but essential steps. Your first priority should be to build a basic online presence — this includes creating a company website, registering on Google Business, and setting up professional social media profiles (like Facebook or LinkedIn). These actions will improve your company's visibility and trustworthiness.

We also recommend using cloud-based email and productivity tools, such as Gmail with Google Workspace or Outlook with Microsoft 365. These platforms make communication and task organization much easier and more efficient.

These are low-cost, easy-to-use technologies that will help you lay a strong foundation for your digital transformation journey.`);
    }
    else if (total < 50) {
        recommendations.push(currentLang === 'pt' ? `Pelas suas respostas, percebemos que a sua empresa já iniciou o percurso digital, mas ainda de forma limitada e pouco integrada. Há presença online, mas muitos dos processos continuam dependentes de métodos tradicionais e manuais.

A nossa sugestão é avançar para ferramentas que melhorem a organização e eficiência interna. Recomendamos, por exemplo, o uso de armazenamento na cloud (como Google Drive ou OneDrive), para facilitar o acesso e partilha de ficheiros. Além disso, ferramentas como o Trello ou o Asana podem ajudá-lo a organizar tarefas e projetos de forma mais clara e colaborativa.

Para melhorar a comunicação entre equipas, vale a pena experimentar o Microsoft Teams ou o Slack. Também recomendamos digitalizar o processo de faturação e contabilidade com ferramentas como o InvoiceXpress ou o Sage.

Estas tecnologias são práticas, intuitivas e não exigem grandes investimentos. Vão permitir-lhe reduzir erros, ganhar tempo e preparar-se para passos mais avançados.` : 
`Your responses show that your company has already taken some steps toward digitalization, but usage is still limited and not fully integrated. There may be some online presence, but many internal processes still rely heavily on traditional, manual methods.

We suggest moving forward by adopting tools that will help improve internal organization and efficiency. For example, cloud storage platforms like Google Drive or OneDrive will allow your team to access and share documents more easily. Task and project management tools, such as Trello or Asana, can also bring structure and clarity to your work processes.

To enhance internal communication, we recommend trying tools like Microsoft Teams or Slack. Additionally, digitizing your invoicing and accounting using software like InvoiceXpress or Sage will save time, reduce errors, and improve control over your finances.

These solutions are practical and accessible and will help your business work smarter and prepare for the next stage of digital growth.`);
    }
    else if (total < 80) {
        recommendations.push(currentLang === 'pt' ? `A sua empresa já utiliza várias ferramentas digitais e demonstra abertura à inovação, o que é um excelente ponto de partida. No entanto, ainda falta uma estratégia integrada que una todos esses recursos.

Neste estágio, recomendamos a adoção de soluções que ajudem a consolidar e escalar os seus processos. Um sistema de CRM como o HubSpot ou o Zoho CRM pode ajudá-lo a organizar contactos, acompanhar vendas e melhorar a relação com clientes. Em paralelo, sugerimos investir em automação de marketing, com ferramentas como o Mailchimp ou o RD Station, para tornar as suas campanhas mais eficazes e profissionais.

Se ainda não o fez, vale a pena considerar um ERP simplificado, como o Moloni ou o PHC GO, para unificar áreas como inventário, faturação, compras e recursos humanos.

Finalmente, recomendamos que comece a explorar análises de dados, com ferramentas como o Google Analytics ou uma versão inicial do Power BI, para tomar decisões mais informadas com base em métricas reais.

Com estas tecnologias, a sua empresa pode dar um salto qualitativo, tornando os processos mais coesos, eficientes e preparados para crescer.` : 
`Your company is already using several digital tools and shows openness to innovation, which is a great sign. However, your systems may still be working independently, without a unified digital strategy.

At this stage, we recommend implementing technologies that help consolidate and scale your operations. A CRM system, such as HubSpot or Zoho CRM, can help you manage customer relationships more effectively and automate your sales processes. In addition, marketing automation tools like Mailchimp or RD Station can boost the performance of your campaigns and help you communicate more strategically with your audience.

You should also consider using a lightweight ERP system, like Moloni or PHC GO, to integrate your company’s departments — from inventory and invoicing to HR and purchasing — in one place.

Finally, we recommend starting to use basic data analytics tools like Google Analytics or a beginner-friendly version of Power BI. These can give you valuable insights into customer behavior, business performance, and operational efficiency.

With these tools, your business can move toward greater integration, improved decision-making, and more sustainable growth.`);
    }
    else {
        recommendations.push(currentLang === 'pt' ? `Parabéns! As suas respostas indicam que a sua empresa já atingiu um nível elevado de maturidade digital. Já existem processos estruturados, uso de tecnologias relevantes e uma cultura favorável à inovação.

Neste momento, o desafio é evoluir ainda mais, apostando em soluções inteligentes, automáticas e que gerem vantagem competitiva. Recomendamos que explore o uso de inteligência artificial (IA) e machine learning aplicados, por exemplo, para prever comportamentos de clientes, automatizar atendimento com chatbots ou otimizar vendas.

Para apoiar decisões estratégicas, sugerimos o uso de plataformas de business intelligence avançadas, como o Power BI Pro ou o Tableau, que permitem análises preditivas com dashboards interativos e completos.

Se atua em áreas como logística, produção ou manutenção, a adoção de tecnologias IoT (Internet das Coisas) pode oferecer uma vantagem relevante — sensores e dispositivos conectados que monitorizam ativos em tempo real. Por fim, pode considerar a automação robótica de processos (RPA) para eliminar tarefas repetitivas e libertar as suas equipas para atividades de maior valor.

A sua empresa já construiu uma base tecnológica forte — agora é o momento de transformar isso em diferenciação e escalabilidade.` : 
`Congratulations! Your responses show that your company has already reached a high level of digital maturity. You have structured processes, relevant digital tools in place, and a strong culture of innovation.

Now it’s time to go even further by implementing intelligent, scalable, and differentiating technologies. We recommend exploring AI and machine learning solutions, such as sales forecasting tools, automated customer service chatbots, or recommendation engines to personalize your customer experience.

To support strategic decision-making, we suggest adopting advanced business intelligence platforms, like Power BI Pro or Tableau. These tools offer interactive dashboards and predictive analytics that help you act on data in real time.

If your company operates in logistics, manufacturing, or maintenance, IoT (Internet of Things) solutions may also be valuable — for example, using connected sensors to monitor equipment and processes in real time. Lastly, we recommend exploring robotic process automation (RPA) tools to eliminate repetitive tasks and free up your team for more strategic work.

You’ve built a solid digital foundation — now is the time to innovate boldly and scale with confidence.

`);
    }

    const radial = `
        <div class="radial-container">
            <div class="radial-progress">
                <div class="radial-fill" style="--progress: ${percentage};"></div>
            </div>
            <div class="radial-percent">${percentage}%</div>
            <p><strong>${message}</strong></p>
            <p>${translations[currentLang].recommendations}</p>
            <ul>${recommendations.map(r => `<li>${r}</li>`).join('')}</ul>
        </div>
    `;

    document.getElementById("result-text").innerHTML = radial;
}


const formQuestions = [
    {
        id: "q0",
    text: { en: "Does your company have a digital strategy?", pt: "A sua empresa tem uma estratégia digital?"},
    options: [
      { text: { en: "No", pt: "Não" }, value: 0 },
      { text: { en: "Partially", pt: "Parcialmente" }, value: 2 },
      { text: { en: "Yes", pt: "Sim" }, value: 4 }
    ]
  },
    {
        id: "q1",
        text: { en: "What are the barriers to having a strategy?", pt: "Quais são os obstáculos à adoção de uma estratégia?" },
        options: [
            { text: { en: "Unclear ROI", pt: "ROI pouco claro"}, value: 2 },
            { text: { en: "Budget constraints", pt: "Restrições orçamentais"} , value: 2 },
            { text: { en: "Leadership resistance", pt: "resistência da liderança"}, value: 1 },
            { text: { en: "Lack of awareness", pt: "Falta de sensibilização"}, value: 1 },
            { text: { en: "Data security and privacy", pt: "Segurança e privacidade dos dados"}, value: 3 }
        ],
        multi: true
    },
    {
        id: "q3",
        text: { en: "How do you perceive digital transformation?", pt: "Como é que encara a transformação digital?" },
        options: [
            { text: { en: "Strategic opportunity", pt: "Oportunidade estratégica"}, value: 5 },
            { text: { en: "Necessary adaptation", pt: "Adaptação necessária"}, value: 3 },
            { text: { en: "Uncertain benefit", pt: "Benefício incerto"}, value: 1 },
            { text: { en: "Risk and cost concern", pt: "Preocupação com os riscos e os custos"},  value: 0 }
        ]
    },
    {
        id: "q4",
        text: { en: "Which technologies are currently in use?", pt: "Que tecnologias estão atualmente a ser utilizadas?"},
        options: [
            { text: { en: "Cloud storage", pt: "Armazenamento cloud"}, value: 1 },
            { text: { en: "AI tools", pt: "Ferramentas AI"}, value: 1 },
            { text: { en: "Process automation", pt: "Automatização de processos"}, value: 1 },
            { text: { en: "Cybersecurity", pt: "Cibersegurança"}, value: 1 },
            { text: { en: "IoT", pt: "IoT"}, value: 1 },
            { text: { en: "Blockchain", pt: "Blockchain"}, value: 1 },
            { text: { en: "Big data", pt: "Big data"}, value: 1 }
        ],
        multi: true
    },
    {
        id: "q5",
        text: {en: "What phase of transformation are you in?", pt: "Em que fase de transformação se encontra?"},
        options: [
            { text: { en: "Initial exploration", pt: "Exploração inicial"}, value: 1 },
            { text: { en: "Planning", pt: "Planeamento"}, value: 2 },
            { text: { en: "Implementation", pt: "Implementação"}, value: 3 },
            { text: { en: "Advanced adoption", pt: "Adoção avançada"}, value: 5 }
        ]
    },
    {
        id: "q6",
        text: { en: "Do you have a dedicated transformation team?", pt: "Dispõe de uma equipa de transformação dedicada?"},
        options: [
            { text: { en: "No", pt: "Não" }, value: 0 },
            { text: { en: "Yes", pt: "Sim" }, value: 5 }
        ]
    },
    {
        id: "q7",
        text: { en: "Have you automated any internal processes?", pt: "Automatizou algum processo interno?"},
        options: [
            { text: { en: "No", pt: "Não" }, value: 0 },
            { text: { en: "Yes", pt: "Sim" }, value: 5 }
        ]
    },
    {
        id: "q8",
        text: { en: "Which areas are automated?", pt: "Que áreas são automatizadas?"},
        options: [
            { text: { en: "Finance", pt:"Finanças"}, value: 1 },
            { text: { en: "Inventory", pt:"Inventário"}, value: 1 },
            { text: { en: "HR", pt: "Recursos Humanos"}, value: 1 },
            { text: { en: "CRM", pt: "CRM"}, value: 1 },
            { text: { en: "Sales", pt: "Vendas"}, value: 1 }
        ],
        multi: true
    },
    {
        id: "q9",
        text: { en: "Are there plans to digitize other areas?", pt: "Há planos para digitalizar outras áreas?"},
        options: [
            { text: { en: "No", pt: "Não" }, value: 0 },
            { text: { en: "Yes", pt: "Sim" }, value: 5 }
        ]
    },
    {
        id: "q10",
        text: { en: "What is the biggest barrier to automation?", pt: "Qual é o maior obstáculo à automatização?"},
        options: [
            { text: { en: "Integration complexity", pt: "Complexidade da integração"}, value: 3 },
            { text: { en: "Lack of knowledge", pt: "Falta de conhecimento"}, value: 1 },
            { text: { en: "High costs", pt: "Custos elevados"}, value: 2 }
        ]
    },
    {
        id: "q11",
        text: { en: "Is there a cybersecurity strategy in place?", pt: "Existe uma estratégia de cibersegurança?"},
        options: [
            { text: { en: "No", pt: "Não" }, value: 0 },
            { text: { en: "Yes", pt: "Sim" }, value: 5 }
        ]
    },
    {
        id: "q12",
        text: { en: "Do employees have adequate digital skills?", pt: "Os empregados têm competências digitais adequadas?"},
        options: [
            { text: { en: "No", pt: "Não" }, value: 0 },
            { text: { en: "Yes", pt: "Sim" }, value: 5 }
        ]
    },
    {
        id: "q13",
        text: { en: "How are employee digital skills developed?", pt: "Como são desenvolvidas as competências digitais dos trabalhadores?"},
        options: [
            { text: { en: "Regular training", pt: "Formação regular"}, value: 5 },
            { text: { en: "Ad-hoc training", pt: "Formação ad-hoc"}, value: 3 },
            { text: { en: "No training", pt: "Sem formação"}, value: 0 }
        ]
    },
    {
        id: "q14",
        text: { en:"How frequent is digital communication in your team?", pt:"Qual a frequência da comunicação digital na sua equipa?"},
        options: [
            { text: { en: "Regular", pt: "Regular"}, value: 5 },
            { text: { en: "Occasional", pt: "Ocasional"}, value: 3 },
            { text: { en: "Rare", pt: "Rara"}, value: 0 }
        ]
    },
    {
        id: "q15",
        text: { en: "Do you engage digitally with customers?", pt: "Interage digitalmente com os clientes?"},
        options: [
            { text: { en: "No", pt: "Não" }, value: 0 },
            { text: { en: "Yes", pt: "Sim" }, value: 5 }
        ]
    },
    {
        id: "q16",
        text: { en: "Which digital channels are you using to engage customers?", pt: "Que canais digitais está a utilizar para envolver os clientes?"},
        options: [
            { text: { en: "Social media", pt: "Redes sociais"}, value: 1 },
            { text: { en:"Email marketing", pt: "Email marketing"}, value: 1 },
            { text: { en: "E-commerce", pt: "E-commerce"}, value: 1 },
            { text: { en: "Chatbots", pt: "Chatbots"},  value: 1 }
        ],
        multi: true
    },
    {
        id: "q17",
        text: { en: "What are your digital transformation goals?", pt: "Quais são os seus objectivos de transformação digital?"},
        options: [
            { text: { en: "Customer experience", pt: "Experiência do cliente"}, value: 1 },
            { text: { en: "Efficiency", pt: "Eficiência"}, value: 1 },
            { text: { en: "New models", pt:"Novos modelos"}, value: 1 },
            { text: { en: "Sustainability", pt: "Sustentabilidade"}, value: 1 },
            { text: { en: "Compliance", pt:"Cumprimento da legislação"}, value: 1 }
        ],
        multi: true
    },
    {
        id: "q18",
        text: { en: "Do you measure the ROI of your digital investments?", pt: "Mede o ROI dos seus investimentos digitais?"},
        options: [
            { text: { en: "No", pt: "Não" }, value: 0 },
            { text: { en: "Yes", pt: "Sim" }, value: 5 }
        ]
    },
    {
        id: "q19",
        text: { en: "How supportive is leadership towards digital transformation?", pt: "Qual é o grau de apoio da liderança à transformação digital?"},
        options: [
            { text: { en: "Strongly supportive", pt: "Apoio total"}, value: 5 },
            { text: { en: "Moderately supportive", pt: "Apoio moderado"}, value: 3 },
            { text: { en: "Neutral or resistant", pt: "Neutro ou resistente"}, value: 0 }
        ]
    },
    {
        id: "q20",
        text: { en: "Have you invested in training/upskilling for digital tools?", pt: "Investiu em formação/aprofundamento de competências em ferramentas digitais?"},
        options: [
            { text: { en: "No", pt: "Não" }, value: 0 },
            { text: { en: "Yes", pt: "Sim" }, value: 5 }
        ]
    },
    {
        id: "q21",
        text: { en: "Are your systems integrated (ERP, CRM, etc.)?", pt: "Os seus sistemas estão integrados (ERP, CRM, etc.)?"},
        options: [
            { text: { en: "No", pt: "Não" }, value: 0 },
            { text: { en: "Yes", pt: "Sim" }, value: 5 }
        ]
    },
    {
        id: "q22",
        text: { en: "Do you use digital tools for data-driven decision-making?", pt: "Utiliza ferramentas digitais para tomar decisões baseadas em dados?"},
        options: [
            { text: { en: "No", pt: "Não" }, value: 0 },
            { text: { en: "Yes", pt: "Sim" }, value: 5 }
        ]
    }
];

</script>
"""))
